In [134]:
""" Third take on machine learning features... 

CURRENTLY TESTING:
(MINIMAL UP-FRONT COMPUTATION)
1. Counts on all intervals/steps (NOT pre-seeding data with the pattern to look outfor
step pattern of 0,3,7 for minor, vs. 0,4,7 for major)
2. Counts of note frequency 


Consider using Irish folkmusic from Music21 corpus... single stream!
"""

###############################################

import music21
import pandas as pd
import numpy as np
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [179]:
training_data = []

# read test files and construct columns
filenames = open('test_files.txt').read().split('\n')

# Instantiating empty list for each feature (note freq and step freq) and the outcome (mode); 
notes_corpus = []
steps_corpus = []
mode_corpus = []

# Iterating through every other file; for early trials using 1/2 files as training set, and other 1/2 to test
for filename in filenames[::2]:
    
    # Create empty score_instance to hold data of training features + outcome
    score_features = {}

    # Convert MIDI file to score and iterate over notes in score 
    # Save relevant individual Note attributes into list, including Note obj (used to calc interval) and name
    score = music21.converter.parse('MIDI test files/Cello solos/' + filename)
    note_attributes = []
    for note in music21.alpha.theoryAnalysis.theoryAnalyzer.getNotes(score, 0):
        if note == None:
            pass
        else:
#             measure = int(note.offset)
#             key = music21.alpha.theoryAnalysis.theoryAnalyzer.getKeyAtMeasure(score, measure)
            note_attributes.append([note, note.name])
    print "Note attributes for file {} completed".format(filename)
            
    # Creates string of all notes in a score and appends to notes_corpus
    all_notes = ""
    for note in note_attributes:
        all_notes += note[1] + " "

    notes_corpus.append(all_notes)
    
    # Creates string of all steps in a score and appends to steps_corpus
    all_steps = ""
    for i in range(1, len(note_attributes)):
        interval = music21.interval.Interval(noteStart=note_attributes[i-1][0], noteEnd=note_attributes[i][0])
        step = int((interval.cents)/100)
        all_steps += str(step) + ' '
    
    steps_corpus.append(all_steps)

    # Determines the mode (major or minor) and assigns to output corpus
    # Note: At the moment, this doesn't take into account changes b/w major + minor w/in a score
    mode_at_measure_0 = music21.alpha.theoryAnalysis.theoryAnalyzer.getKeyAtMeasure(score, 0).mode
    is_major = (mode_at_measure_0 == 'major')
    mode_corpus.append(is_major)


# Creates weighted vector of note frequency for each score
notes_vectorizer = TfidfVectorizer(min_df=1, analyzer='word', stop_words=None, token_pattern=r'\w#?-?')
note_freq_vectors = notes_vectorizer.fit_transform(notes_corpus).todense()
print note_freq_vectors

# Creates weighted vector of step frequency for each score
steps_vectorizer = TfidfVectorizer(min_df=1, analyzer='word', stop_words=None, ngram_range=(2,3), token_pattern=r'\d\d?')
step_freq_vectors = steps_vectorizer.fit_transform(steps_corpus).todense()
print step_freq_vectors

# Zips both features together along with outcome (mode)
zipped_features = zip(note_freq_vector, step_freq_vector, mode_corpus)
print zipped_features


Note attributes for file cs1-1pre.mid completed
Note attributes for file cs1-3cou.mid completed
Note attributes for file cs1-5men.mid completed
Note attributes for file cs2-1pre.mid completed
Note attributes for file cs2-3cou.mid completed
Note attributes for file cs2-5men.mid completed
Note attributes for file cs3-1pre.mid completed
Note attributes for file cs3-3cou.mid completed
Note attributes for file cs3-5bou.mid completed
Note attributes for file cs4-1pre.mid completed
Note attributes for file cs4-3cou.mid completed
Note attributes for file cs4-5bou.mid completed
Note attributes for file cs5-1pre.mid completed
Note attributes for file cs5-3cou.mid completed
Note attributes for file cs5-5gav.mid completed
Note attributes for file cs6-1pre.mid completed
Note attributes for file cs6-3cou.mid completed
Note attributes for file cs6-5gav.mid completed
[[ 0.37589204  0.36197011  0.01467465  0.31556368  0.08804788  0.37125139
   0.30816759  0.02784385  0.06188161  0.43317127  0.44086103 

In [74]:
# Create dataframe to hold all relevant training feature and outcome data
# Incl: 'note_count', 'step_count', 'is_major'
# df = None

# for i in range(len(training_data)):
# #     print training_data[i]['note_count']

#     note_count_df = pd.DataFrame(training_data[i]['note_count'], index=[0], dtype='float64')
#     step_count_df = pd.DataFrame(training_data[i]['step_count'], index=[0], dtype='float64')
#     outcome_df = pd.Series(training_data[i]['is_major'], name='is_major')

#     instance_df = pd.concat([note_count_df, step_count_df, outcome_df], axis=1)
#     if i == 0:
#         df = instance_df
#     else:
#         df = pd.concat([df, instance_df], axis=0, ignore_index=True)



In [71]:
# Plot training data, O = major outcome, x = minor outcome
# is_major_filter = (df['is_major'] == True)
# is_minor_filter = (df['is_major'] == False)

# plt.plot(df[is_major_filter],
#          linestyle='', 
#          marker='o')

# plt.plot(df[is_minor_filter], 
#          linestyle='', 
#          marker='x')

# plt.xlabel('Step')
# plt.ylabel('Step Frequency')

In [159]:
# # Plot training data, O = major outcome, x = minor outcome
# is_major_filter = (df['is_major'] == True)
# is_minor_filter = (df['is_major'] == False)

# plt.plot(df[is_major_filter], 
#          linestyle='', 
#          marker='o')

# plt.plot(df[is_minor_filter], 
#          linestyle='', 
#          marker='x')

# plt.xlabel('Notes')
# plt.ylabel('Note Frequency')


In [ ]:
# NEXT STEPS: 
# 1. Look up how to pull data from dictionaries into pd dataframe
# 2. Explore Patsy, is this necessary intermediate between pds and scikit-learn?
# 3. Anything more to declare for classifier and params before running logistic reg

In [174]:
X1_train = note_freq_vectors
y1_train = np.ravel(mode_corpus)

# for song in training_data[:19]:
#     X_train.append((song['note_count'], song['step_count']))
#     y_train.append(song['is_major'])
    
# # Creating classifiers
lr = LogisticRegression()

# # Fit model to the training data
notes_lr = lr.fit(X1_train, y1_train)
notes_lr.score(X1_train, y1_train)

0.77777777777777779

In [175]:
notes_testing = []
mode_testing = []

for filename in filenames[1::2]:
    
    # Convert MIDI file to score and iterate over notes in score 
    # Save relevant individual Note attributes into list, including Note obj (used to calc interval) and name
    score = music21.converter.parse('MIDI test files/Cello solos/' + filename)
    note_attributes = []
    for note in music21.alpha.theoryAnalysis.theoryAnalyzer.getNotes(score, 0):
        if note == None:
            pass
        else:
#             measure = int(note.offset)
#             key = music21.alpha.theoryAnalysis.theoryAnalyzer.getKeyAtMeasure(score, measure)
            note_attributes.append([note, note.name])
    print "Note attributes for file {} completed".format(filename)
            
    # Creates string of all notes in a score and appends to notes_corpus
    all_notes = ""
    for note in note_attributes:
        all_notes += note[1] + " "

    notes_testing.append(all_notes)

    # Determines the mode (major or minor) and assigns to output corpus
    # Note: At the moment, this doesn't take into account changes b/w major + minor w/in a score
    mode_at_measure_0 = music21.alpha.theoryAnalysis.theoryAnalyzer.getKeyAtMeasure(score, 0).mode
    is_major = (mode_at_measure_0 == 'major')
    mode_testing.append(is_major)

# Creates weighted vector of note frequency for each score
notes_vectorizer = TfidfVectorizer(min_df=1, analyzer='word', stop_words=None, token_pattern=r'\w#?-?')
note_freq_testing = notes_vectorizer.fit_transform(notes_testing).todense()

for i in range(len(note_freq_testing)):
    print notes_lr.predict(note_freq_testing[i])
    print mode_testing[i]


Note attributes for file cs1-2all.mid completed
Note attributes for file cs1-4sar.mid completed
Note attributes for file cs1-6gig.mid completed
Note attributes for file cs2-2all.mid completed
Note attributes for file cs2-4sar.mid completed
Note attributes for file cs2-6gig.mid completed
Note attributes for file cs3-2all.mid completed
Note attributes for file cs3-4sar.mid completed
Note attributes for file cs3-6gig.mid completed
Note attributes for file cs4-2all.mid completed
Note attributes for file cs4-4sar.mid completed
Note attributes for file cs4-6gig.mid completed
Note attributes for file cs5-2all.mid completed
Note attributes for file cs5-4sar.mid completed
Note attributes for file cs5-6gig.mid completed
Note attributes for file cs6-2all.mid completed
Note attributes for file cs6-4sar.mid completed
Note attributes for file cs6-6gig.mid completed
[ True]
True
[ True]
True
[ True]
True
[ True]
False
[False]
False
[False]
False
[ True]
True
[ True]
True
[ True]
True
[False]
False
[F

In [180]:
X2_train = step_freq_vectors
y2_train = np.ravel(mode_corpus)

# for song in training_data[:19]:
#     X_train.append((song['note_count'], song['step_count']))
#     y_train.append(song['is_major'])
    
# # Creating classifiers
lr = LogisticRegression()

# # Fit model to the training data
notes_lr = lr.fit(X2_train, y2_train)
notes_lr.score(X2_train, y2_train)

0.77777777777777779

In [181]:
steps_testing = []
mode_testing = []

for filename in filenames[1::2]:
    
    # Convert MIDI file to score and iterate over notes in score 
    # Save relevant individual Note attributes into list, including Note obj (used to calc interval) and name
    score = music21.converter.parse('MIDI test files/Cello solos/' + filename)
    note_attributes = []
    for note in music21.alpha.theoryAnalysis.theoryAnalyzer.getNotes(score, 0):
        if note == None:
            pass
        else:
#             measure = int(note.offset)
#             key = music21.alpha.theoryAnalysis.theoryAnalyzer.getKeyAtMeasure(score, measure)
            note_attributes.append([note, note.name])
    print "Note attributes for file {} completed".format(filename)
            
    # Creates string of all steps in a score and appends to steps_corpus
    all_steps = ""
    for i in range(1, len(note_attributes)):
        interval = music21.interval.Interval(noteStart=note_attributes[i-1][0], noteEnd=note_attributes[i][0])
        step = int((interval.cents)/100)
        all_steps += str(step) + ' '
    
    steps_testing.append(all_steps)
    
    # Determines the mode (major or minor) and assigns to output corpus
    # Note: At the moment, this doesn't take into account changes b/w major + minor w/in a score
    mode_at_measure_0 = music21.alpha.theoryAnalysis.theoryAnalyzer.getKeyAtMeasure(score, 0).mode
    is_major = (mode_at_measure_0 == 'major')
    mode_testing.append(is_major)

# Creates weighted vector of note frequency for each score
steps_vectorizer = TfidfVectorizer(min_df=1, analyzer='word', stop_words=None, ngram_range=(2,3), token_pattern=r'\w#?-?')
step_freq_testing = steps_vectorizer.fit_transform(steps_testing).todense()

for i in range(len(step_freq_testing)):
    print notes_lr.predict(step_freq_testing[i])
    print mode_testing[i]


Note attributes for file cs1-2all.mid completed
Note attributes for file cs1-4sar.mid completed
Note attributes for file cs1-6gig.mid completed
Note attributes for file cs2-2all.mid completed
Note attributes for file cs2-4sar.mid completed
Note attributes for file cs2-6gig.mid completed
Note attributes for file cs3-2all.mid completed
Note attributes for file cs3-4sar.mid completed
Note attributes for file cs3-6gig.mid completed
Note attributes for file cs4-2all.mid completed
Note attributes for file cs4-4sar.mid completed
Note attributes for file cs4-6gig.mid completed
Note attributes for file cs5-2all.mid completed
Note attributes for file cs5-4sar.mid completed
Note attributes for file cs5-6gig.mid completed
Note attributes for file cs6-2all.mid completed
Note attributes for file cs6-4sar.mid completed
Note attributes for file cs6-6gig.mid completed


ValueError: X has 706 features per sample; expecting 1812

In [1]:
i

NameError: name 'i' is not defined